# **NER 사전 만들기**
https://pypi.org/project/mdict-utils/
1. **MDX 사전파일을** 활용하여 객체에 구분하기

## **1 Sqlite3 to Pandas**
MDX 사전파일을 Sqlite3 로 변경한 뒤, 작업에 유용한 형태로 전처리 하기

In [5]:
import sqlite3
import pandas as pd
# Create your connection.
conn = sqlite3.connect('backup/kordict.db')
resp = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tableNames = [name[0]  for name in resp]
tableNames

['meta', 'mdx']

In [6]:
%%time
df = pd.read_sql_query("SELECT * FROM {}".format(tableNames[1]), conn)
df.head()

CPU times: user 1.18 s, sys: 140 ms, total: 1.32 s
Wall time: 1.32 s


,entry,paraphrase
0,ᄛ,<font color=crimson size=+2><b>ᄛ</b></font><br...
1,ᄻ,<font color=crimson size=+2><b>ᄻ</b></font><br...
2,ᆐ,<font color=crimson size=+2><b>ᆐ</b></font><br...
3,ㄱ,<font color=crimson size=+2><b>ㄱ</b></font><br...
4,ㄱ,<font color=crimson size=+2><b>ㄱ</b></font><br...


## **2 사전파일 살펴보기**
**paraphrase** 내용을 **사용자가 필요한 형태로** 변경하기
```html
<font color=crimson size=+2>
    <b>ㄹ라</b>
</font>
<br>
<br>
<font color="#00008b">
    <b>-ㄹ라03</b>
</font>
<br>
<font color="#549606">
    <b>「어미」</b>
</font>
<br>『방언』<br>‘-려04’의 방언(경상).\r\n
```

In [7]:
# 색인정보 : 51만개의 단어사전 데이터
itemIndex = df.entry.values.tolist()
itemData  = df.paraphrase.values.tolist()
len(itemIndex), itemIndex[:10]

(516490, ['ᄛ', 'ᄻ', 'ᆐ', 'ㄱ', 'ㄱ', 'ㄱㄴㄷ순', 'ㄱㄴㄷ차례', 'ㄱㄴ순', 'ㄱㄴ차례', 'ㄱ자'])

In [8]:
data = itemData[0]
data = data.lower()  # html Tag 전처리
print(data)

# 간단하게 처리한 뒤 유사내용 묶기
from lxml.html import fromstring
fromstring(data).xpath('//text()')

<font color=crimson size=+2><b>ᄛ</b></font><br><br><font color="#00008b"><b>ᄛ</b></font>『옛말』<br>『언어』<br>한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.



['ᄛ',
 'ᄛ',
 '『옛말』',
 '『언어』',
 '한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.\r\n']

## **3 Regex를 활용하여 구분하기**
본문에 "\<br\>" 내용이 많아서 "//text()" 를 사용하면 컬럼갯수가 꼬여버린다
regex, lxml 을 사용하면 좋겠지만 경우의 수가 많아서 모두 처리하지 않으면 안됨..

In [79]:
data = itemData[2945]
data = data.lower()  # html Tag 전처리
data

'<font color=crimson size=+2><b>가변표지</b></font><br><br><font color="#00008b"><b>가변^표지(可變標識)</b></font>『교통』<br>교통에 대한 규제나 경고, 안내 따위의 내용을 시간별로 또는 원격 조정 신호에 따라 자동적으로 변경하는 표지.\r\n'

In [80]:
import re
re.findall(r'<font color=crimson (.*?)</font>', data, re.DOTALL)

[' size=+2><b>가변표지</b>']

In [85]:
import re
# html 태그내용 제거하는 함수
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [84]:
remove_html_tags(data)

re.compile('<.*?>')


'가변표지가변^표지(可變標識)『교통』교통에 대한 규제나 경고, 안내 따위의 내용을 시간별로 또는 원격 조정 신호에 따라 자동적으로 변경하는 표지.\r\n'

In [57]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(data, "lxml")
bs_dom = dom.find_all('font')
bs_txt = [str(_.get_text()) for _ in bs_dom]
bs_txt

['가변표지', '가변^표지(可變標識)']

In [68]:
bs_result = list(map(lambda x : str(x),  bs_dom))
bs_result[0]

'<font color="crimson" size="+2"><b>가변표지</b></font>'

In [74]:
bs_result = list(map(lambda x : str(x),  bs_dom))
bs_result[1]

'<font color="#00008b"><b>가변^표지(可變標識)</b></font>'

In [77]:
re.sub('color="#00008b">', "", data)

'<font color=crimson size=+2><b>가변표지</b></font><br><br><font <b>가변^표지(可變標識)</b></font>『교통』<br>교통에 대한 규제나 경고, 안내 따위의 내용을 시간별로 또는 원격 조정 신호에 따라 자동적으로 변경하는 표지.\r\n'

In [63]:
re.sub("<font", "", data)

' color=crimson size=+2><b>가변표지</b></font><br><br> color="#00008b"><b>가변^표지(可變標識)</b></font>『교통』<br>교통에 대한 규제나 경고, 안내 따위의 내용을 시간별로 또는 원격 조정 신호에 따라 자동적으로 변경하는 표지.\r\n'

In [20]:
"".join(dom.find_all('font'))

TypeError: sequence item 0: expected str instance, Tag found

In [14]:
import re # 강결합으로 정치큰 부분 제거하기
re.sub('<br><font color=.*</font>', "",data)

'<font color=crimson size=+2><b>ᄛ</b></font><br>『옛말』<br>『언어』<br>한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.\r\n'

In [8]:
_ = re.sub('<font .*</font><br>', "",data).split("<br>")
_[0], _[1].replace("\r\n", "")

('', '<font color="#00008b"><b>ᄛ</b></font>『옛말』')

In [9]:
data

'<font color=crimson size=+2><b>ᄛ</b></font><br><br><font color="#00008b"><b>ᄛ</b></font>『옛말』<br>『언어』<br>한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.\r\n'

In [ ]:
re.findall(r"", )

## **3 함수의 적용**
작업용 함수를 작성한 뒤 응용하기

In [4]:
from lxml.html import fromstring
def exportTxt(data):
    yield fromstring(data).xpath("//text()")

# result = list(map(lambda x: next(exportTxt(x)), itemData[:200000]))
# result = list(map(lambda x: next(exportTxt(x)), itemData[200000:400000]))
result = list(map(lambda x: next(exportTxt(x)), itemData[400000:]))

import pickle
# Store data (serialize)
with open('data3.pk', 'w') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## **4 Pickle 내용 합치기**
작업용 함수를 작성한 뒤 응용하기

In [7]:
from glob import glob
pklist = sorted(glob('data?.pk'))

# Load data (deserialize)
import pickle
result = []
for _ in pklist:
    with open(_, 'rb') as handle:
        result.extend(pickle.load(handle))

In [11]:
with open('data.pk', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

## **5 사전자료 전처리 작업**
작업용 함수를 작성한 뒤 응용하기

In [24]:
with open('data.pk', 'rb') as handle:
    mdkList = pickle.load(handle)

mdkList =[ list(map(lambda x:str(x), _)) for _ in mdkList]
len(mdkList)

In [30]:
mdkList[3][-2]

'『언어』'

In [31]:
# tag 내용만 추출하기
tags = [_[-2]  for _ in mdkList]
len(set(tags))

121348

In [32]:
list(set(tags))

['어떤 한 곳이나 일에 관심을 집중하여 기울이다.',
 '갖가지 약. ',
 '‘부인01(夫人)’의 옛말. ',
 '다채롭다. 세미하다03. ',
 '빗줄기나 물줄기 따위가 굵고 세찬 것을 비유적으로 이르는 말.',
 '조금 휘어져 뒤로 자빠질 듯 비스듬하다.',
 '부녀자들로 구성된 모임. 흔히 마을ㆍ동(洞)ㆍ아파트 따위의 인근 지역 사회라든가 종교 단체 따위에서, 부녀자들이 모임을 구성하여 아이들의 교육이나 일상생활 문제 따위에 공동으로 대처한다. ',
 '‘좁쌀’의 옛말. ',
 '무인으로서의 운수. ',
 '임금이 나라 안을 두루 살피며 돌아다니다. ≒순공하다01ㆍ순행하다02.',
 '‘출사하다01(出仕―)’의 옛말. ',
 '적의 사격이나 관측으로부터 아군을 보호하기 위하여 땅을 파서 만든 구덩이. ≒벙커',
 '복잡하게 그려 놓은 그림에 숨겨진 물건을 찾도록 한 놀이. ',
 '열렬히 사랑하다.',
 '특수한 칠감을 발라 어두운 곳에서 빛을 내게 하는 색종이. ≒발광색지. ',
 '앞으로의 일이 어떻게 될 것인가가 미리 정해지다.',
 ' =은퇴하다.',
 '앞으로 닥쳐올 일이 미리 생각되고 기다려지다.',
 '빌려 주다. ',
 '물고기가 한창 알을 낳는 때. ',
 '자연치료 수단을 개발하여 사람의 체력 단련이나 질병의 예방ㆍ치료에 이용하는 것을 연구하는 학문. 임상 의학의 한 분야이다.',
 '진귀한 책. ≒진본01(珍本)ㆍ진적01(珍籍).',
 '촘촘하게 늘어서 있다. ',
 '‘물건’의 뜻을 더하는 접미사.',
 '무농성잇과 살파의 하나. 무성 세대 개체의 길이는 5~12mm, 유성 세대 개체의 길이는 3~6mm이며 항아리 모양이다. 따뜻한 바다에 살면서 해초와 작은 갑각류를 먹는데 바닷물고기의 좋은 먹이가 된다.',
 '【…을】공공의 물건을 사사로이 쓰다.',
 '기쁨이나 분노 따위의 감정이 격렬히 일어나게 되다.',
 '마음씨가 고우면 옷 앞섶이 아문다',
 '헤아릴 수 없다.',
 '얼음이 깔려 있는 길. ',
 '사람을 자기 마음대

In [28]:
import re
re.findall(r"[^ 가-힣]+", mdkDict[:3][0][0])

['ᄛ']

In [6]:
result[:4]

[['ᄛ',
  'ᄛ',
  '『옛말』',
  '『언어』',
  '한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.\r\n'],
 ['ᄻ',
  'ᄻ',
  '『옛말』',
  '한글 옛 자모 ‘ㅎ’의 된소리를 적는 자모. ‘\uec96다’의 ‘\uec96’에서와 같이 초성으로만 쓰였다.\r\n'],
 ['ᆐ', 'ᆐ', '「명사」', '『옛말』', '『언어』', '훈민정음 모음 글자의 하나, ‘\uf31d’의 중성으로 썼다.\r\n'],
 ['ㄱ',
  'ㄱ01 [기역]',
  ':〔ㄱ만[기영-]〕',
  '「명사」',
  '『언어』',
  '한글 자모의 첫째 글자. 기역이라 이르며, ‘가’에서와 같이 초성으로, ‘역’에서와 같이 종성으로 쓰인다. 목젖으로 콧길을 막고 혀뿌리를 높여 연구개를 막았다가 뗄 때 나는 무성음이다. 초성일 때는 무성이나 모음 사이에서는 유성음이 되며, 종성일 때는 혀뿌리를 떼지 않고 발음한다. 혀뿌리가 목구멍을 막는 모양을 본떠서 만든 글자이다.\r\n']]

In [7]:
%whos

Variable     Type          Data/Info
------------------------------------
conn         Connection    <sqlite3.Connection object at 0x7f2f291d6490>
df           DataFrame            entry             <...>[516490 rows x 2 columns]
exportTxt    function      <function exportTxt at 0x7f2f5472d9d8>
fromstring   function      <function fromstring at 0x7f2f274d7c80>
itemData     list          n=516490
itemIndex    list          n=516490
pd           module        <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
resp         Cursor        <sqlite3.Cursor object at 0x7f2f29176dc0>
result       list          n=200000
sqlite3      module        <module 'sqlite3' from '/<...>3.6/sqlite3/__init__.py'>
tableNames   list          n=2


In [ ]:
# http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%A0%9C%EB%84%88%EB%A0%88%EC%9D%B4%ED%84%B0-generator/

In [3]:
def square_numbers(nums):
    for i in nums:
        yield i * i

my_nums = square_numbers([1, 2, 3, 4, 5])  #1
next(my_nums)

1

In [1]:
def foo(a, b):
    if a < 3:
        yield a, b
        a += 1
        for a, v in foo(a, b):
            yield a, v
    else:
        yield a, b

for a, v in foo(1, 2):
    print(a, v)

1 2
2 2
3 2


In [ ]:
itemData

In [10]:
import re # 강결합으로 정치큰 부분 제거하기
re.sub('<br><font .*</font><br>', "",data)

'<font color=crimson size=+2><b>ᄛ</b></font><br><br><font color="#00008b"><b>ᄛ</b></font>『옛말』<br>『언어』<br>한글 옛 자모의 하나. 반설경음이라 이르며, 혀끝이 윗잇몸을 가볍게 두들겨 내는 유음으로 짐작된다.\r\n'

In [8]:
_ = re.sub('<font .*</font><br>', "",data).split("<br>")
_[0], _[1].replace("\r\n", "")

('', '<font color="#00008b"><b>ᄛ</b></font>『옛말』')

In [4]:
import re
from lxml.html import fromstring
def exportMdx(data):
    data = data.lower()  # html Tag 전처리
    _ = re.sub('<font .*</font><br>', "",data).split("<br>")
    return fromstring(data).xpath('//font//text()'), (_[0], _[1].replace("\r\n", ""))

In [5]:
result = []
from tqdm import tqdm
for _ in tqdm(itemData):
    result.append(exportMdx(_))


  0%|          | 0/516490 [00:00<?, ?it/s]


IndexError: list index out of range

In [21]:
def exportTag(data):
    temp = data.lower()
    temp = fromstring(temp).xpath('//font[@color="#549606"]//text()')
    return temp.strip()

from lxml.html import fromstring
from tqdm import tqdm
result = []
for _ in tqdm(itemData):
    result.append(exportTag(_))
tags = list(set(result))

  0%|          | 0/516490 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'strip'

In [15]:
tags

['「조사」']

In [6]:
temp = str(df.entry[0])

In [7]:
temp

'ᄛ'